In [1]:
import pandas as pd
import pyarrow.parquet as pq
import pyarrow as pa
import glob
import os
from tqdm import tqdm
tqdm.pandas()
pd.options.mode.chained_assignment = None

/home/yin/anaconda3/envs/py3/lib/python3.7/site-packages/tqdm/std.py:656: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
df = pd.read_pickle('UK_Oxford_skillCluster_processed_all.pkl')

In [3]:
def select_df(df, start_date, end_date, 
              city = False, sic = False, soc = False, county = False):
    mask = (df['date'] >= start_date) & (df['date'] <= end_date)
    df = df.loc[mask]
    if city != False:
        df = df[(df['CanonCity'] == city)]
    if sic != False:
        df = df[(df['SICCode'].str[:4] == sic)]
    if soc != False:
        df = df[(df['SOCCode'].str[:3] == soc)]
    if county != False:
        df = df[(df['CanonCounty'] == county)]
        
    return df

In [5]:
## 1)
start_date = '2012-01-01'
end_date = '2019-11-04'
county = 'Oxfordshire'

s1 = select_df(df, start_date = start_date
               , end_date = end_date, county = county)
skill_count_df1 = pd.DataFrame(s1['CanonSkillClusters'].value_counts())
skill_count_df1['Percent'] = (skill_count_df1['CanonSkillClusters']/skill_count_df1['CanonSkillClusters'].sum())*100
skill_count_df1


,CanonSkillClusters,Percent
Specialised Skills,829666,13.674314
Specialised Skills|Specialised Skills,228146,3.760236
None,125181,2.063197
Specialised Skills|Sales: General Sales,103741,1.709829
Specialised Skills|Customer and Client Support: Basic Customer Service,96355,1.588095
...,...,...
Specialised Skills|Specialised Skills|Common Skills|Specialised Skills|Specialised Skills|Specialised Skills|Energy and Utilities: Energy Management,1,0.000016
Specialised Skills|Common Skills|Specialised Skills|Specialised Skills|Environment: Conservation,1,0.000016
Specialised Skills|Specialised Skills|Specialised Skills|Common Skills|Specialised Skills|Health Care: Pathology,1,0.000016
Common Skills|Energy and Utilities: Clean Energy,1,0.000016


In [7]:
#2)

s2 = s1.groupby(["CanonSkillClusters", "SICCode"]).size(
).to_frame('Count').sort_values(['Count'],ascending=False).reset_index()

s2['Percent'] = s2.progress_apply(lambda x: 100*x['Count']/s2[(s2['SICCode']==x['SICCode'])]['Count'].sum(), axis=1)
skill_count_df2 = pd.DataFrame(s2)
skill_count_df2

100%|██████████| 223716/223716 [57:43<00:00, 64.59it/s] 


,CanonSkillClusters,SICCode,Count,Percent
0,Specialised Skills,86,47701,16.580233
1,Specialised Skills,C,36832,12.184326
2,Specialised Skills,85.42,26441,12.643090
3,Specialised Skills,85,13908,21.429232
4,Specialised Skills,56.10,11723,18.182523
...,...,...,...,...
223711,Specialised Skills|Common Skills|Specialised S...,56.10,1,0.001551
223712,Specialised Skills|Common Skills|Specialised S...,85.42,1,0.000478
223713,Specialised Skills|Common Skills|Specialised S...,88,1,0.015987
223714,Specialised Skills|Common Skills|Specialised S...,88.99,1,0.002905


In [ ]:
#3)
s3 = s1.groupby(["CanonSkillClusters", "SOCCode"]).size().to_frame('Count').sort_values(['Count'],ascending=False).reset_index()
s3['Percent'] = s3.progress_apply(lambda x: 100*x['Count']/s3[(s3['SOCCode']==x['SOCCode'])]['Count'].sum(), axis=1)
skill_count_df3 = pd.DataFrame(s3)
skill_count_df3

 35%|███▍      | 94645/271499 [31:11<58:01, 50.80it/s]  

In [ ]:
#4)
s4 = s1.groupby(["CanonSkillClusters", "LocalAuthorityDistrict"]).size().to_frame('Count').sort_values(['Count'],ascending=False).reset_index()

s4['Percent'] = s4.progress_apply(lambda x: 100*x['Count']/s4[(s4['LocalAuthorityDistrict']==x['LocalAuthorityDistrict'])]['Count'].sum(), axis=1)
skill_count_df4 = pd.DataFrame(s4)
skill_count_df4

In [ ]:
#5)
s5 = s1.groupby(["CanonSkillClusters", "CanonCity"]).size().to_frame('Count').sort_values(['Count'],ascending=False).reset_index()

s5['Percent'] = s5.progress_apply(lambda x: 100*x['Count']/s5[(s5['CanonCity']==x['CanonCity'])]['Count'].sum(), axis=1)
skill_count_df5 = pd.DataFrame(s5)
skill_count_df5

In [ ]:
## Output to csv
for i in range(1,6):
    eval("skill_count_df"+str(i)).to_csv('skillClusters_count_df' + str(i) + '.csv')



In [ ]:
#6)
start_date = '2012-01-01'
end_date = '2019-11-04'
county = 'Oxfordshire'

s1 = select_df(df, start_date = start_date
               , end_date = end_date, county = county)
s1['year'] = s1['date'].dt.year

skillClusters_count_df6 = pd.DataFrame(s1.groupby(['year', 'CanonSkillClusters']).size().to_frame('count').sort_values(['year', 'count'],ascending=False)).reset_index()
skillClusters_count_df6

## skillClusters_count_df6
skillClusters_count_df6_reshape = skillClusters_count_df6.pivot(columns='year',index='CanonSkillClusters', values='count')

skillClusters_count_df6_reshape.to_csv('skillClusters_count_df6_reshape.csv')

skillClusters_count_df6_reshape.pct_change(axis='columns').to_csv('skillClusters_count_df6_reshape_percent.csv')

In [ ]:
#7)
start_date = '2019-01-01'
end_date = '2019-10-30'
county = 'Oxfordshire'

s1 = select_df(df, start_date = start_date
               , end_date = end_date, county = county)
s1['month'] = s1['date'].dt.month

skillClusters_count_df7 = pd.DataFrame(s1.groupby(['month', 'CanonSkillClusters']).size().to_frame('count').sort_values(['month', 'count'],ascending=False)).reset_index()
skillClusters_count_df7

## skillClusters_count_df6
skillClusters_count_df7_reshape = skillClusters_count_df7.pivot(columns='month',index='CanonSkillClusters', values='count')

skillClusters_count_df7_reshape.to_csv('skillClusters_count_df7_reshape.csv')

skillClusters_count_df7_reshape.pct_change(axis='columns').to_csv('skillClusters_count_df7_reshape_percent.csv')

In [ ]:
#8)
start_date = '2012-01-01'
end_date = '2019-11-04'
county = 'Oxfordshire'

s1 = select_df(df, start_date = start_date
               , end_date = end_date, county = county)
s1['year'] = s1['date'].dt.year

skillClusters_count_df8 = pd.DataFrame(s1.groupby(['year','CanonSkillClusters', 'SOCCode', 'SICCode']).size().to_frame('count').sort_values(['year', 'CanonSkillClusters','SOCCode', 'SICCode'],ascending=False)).reset_index()
skillClusters_count_df8

skillClusters_count_df8 = skillClusters_count_df8.pivot_table(
        values='count', 
        index=['CanonSkillClusters', 'SOCCode', 'SICCode'], 
        columns='year')
skillClusters_count_df8

skillClusters_count_df8.to_csv('skillClusters_count_df8.csv')
skillClusters_count_df8.pct_change(axis='columns').to_csv('skillClusters_count_df8_percent.csv')

In [ ]:
#9)
start_date = '2019-01-01'
end_date = '2019-10-30'
county = 'Oxfordshire'

s1 = select_df(df, start_date = start_date
               , end_date = end_date, county = county)
s1['month'] = s1['date'].dt.month

skillClusters_count_df9= pd.DataFrame(s1.groupby(['month','CanonSkillClusters', 'SOCCode', 'SICCode']).size().to_frame('count').sort_values(['month', 'CanonSkillClusters','SOCCode', 'SICCode'],ascending=False)).reset_index()

skillClusters_count_df9= skillClusters_count_df9.pivot_table(
        values='count', 
        index=['CanonSkillClusters', 'SOCCode', 'SICCode'], 
        columns='month')
skillClusters_count_df9

skillClusters_count_df9.to_csv('skillClusters_count_df9_month.csv')
skillClusters_count_df9.pct_change(axis='columns').to_csv('skillClusters_count_df9_month_percent.csv')

In [ ]:
# 10 & 11)
start_date = '2012-01-01'
end_date = '2019-11-04'
county = 'Oxfordshire'

s1 = select_df(df, start_date = start_date
               , end_date = end_date, county = county)
s1['year'] = s1['date'].dt.year

skillClusters_count_df10 = pd.DataFrame(s1.groupby(['year']).size().to_frame('count').sort_values(['count'],ascending=False)).reset_index()
skillClusters_count_df10.to_csv('skillClusters_count_df10_year.csv')
skillClusters_count_df10

start_date = '2019-01-01'
end_date = '2019-10-30'
county = 'Oxfordshire'

s1 = select_df(df, start_date = start_date
               , end_date = end_date, county = county)
s1['month'] = s1['date'].dt.month
skillClusters_count_df11 = pd.DataFrame(s1.groupby(['month']).size().to_frame('count').sort_values(['count'],ascending=False)).reset_index()
skillClusters_count_df11.to_csv('skillClusters_count_df11_month.csv')
skillClusters_count_df11
